In [17]:
from omegaconf import OmegaConf
from datasets import load_data
import numpy as np
config = OmegaConf.load("./configs/default_config.yaml")
for dataset in ['Imagenette', 'Flowers', 'EuroSAT', 'Aircraft', 'CIFAR10', 'CIFAR100' ]:
    config.dataset = dataset
    train_dataset, val_dataset, test_dataset, N, train_transform, transform = load_data(False, config) 
    image, label = val_dataset[0]  # Extract the image and label
    pixels = np.array(image)
    print(dataset, len(train_dataset), len(val_dataset), len(test_dataset), min(pixels[0][0]),  max(pixels[0][0]), N)

datadir /home/mheuillet/Desktop/robust_training/data
Imagenette 9469 982 2943 39 39 10
Flowers 1020 1020 6149 61 128 102
EuroSAT 18360 3240 5400 138 208 10
Aircraft 3334 3333 3333 111 186 100
Files already downloaded and verified
CIFAR10 47500 2500 10000 121 194 10
CIFAR100 47500 2500 10000 10 52 100
